In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
from pathlib import Path
import re
import sys
import yaml
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from ieee_train import eval_train, eval_check_feature
from kaggle_utils import reduce_mem_usage, move_feature

In [3]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET, 'is_train', 'date']

def filter_feature(path):
    if \
    path.count('') \
    :
#     and not path.count('raw__D') \
#     and not path.count('raw__C') \
#     and not path.count('raw__V') \
#     or path.count('Fraud') 
#     or path.count('tionID') 
#     or path.count('C1')
#     or path.count('C13')
#     or path.count('V232')
#     or path.count('addr1')
        return True
    else:
        return False


paths_train = glob('../feature/eda_base/*_train.gz')
paths_train = [path for path in paths_train if filter_feature(path) ]
df_train = parallel_load_data(paths_train)

group_kfold_path = '../input/0908_ieee__DT-M_GroupKFold.gz'
group = read_pkl_gzip(group_kfold_path)
COLUMN_GROUP = 'DT-M'
df_train[COLUMN_GROUP] = group

In [43]:
#========================================================================
# Target Encoding
#========================================================================

weight_map = {
    '2017-12': 0.25,
    '2018-1' : 0.40,
    '2018-2' : 0.55,
    '2018-3' : 0.70,
    '2018-4' : 0.85,
    '2018-5' : 1.0,
}

cols_card = [col for col in df_train.columns if col.startswith('card')]
print(cols_card)
for col in tqdm(cols_card):
    tmp = df_train.groupby([COLUMN_GROUP,col], as_index=False)[COLUMN_TARGET].agg({
        f'{col}_mean': 'mean'
    })
    tmp_all = df_train.groupby([col], as_index=False)[COLUMN_TARGET].agg({
        f'{col}_all_mean': 'mean'
    })
    merge = tmp_all.merge(tmp, how='left', on=col)

    base = df_train[[col]]
    
    # pd.set_option('max_rows', 100)
    merge['ratio'] = merge[f'{col}_mean'] / merge[f'{col}_all_mean']
    
    merge.reset_index(inplace=True)
    tmp_base = merge[[col, f'{col}_all_mean']].drop_duplicates()
    merge.set_index(col, inplace=True)
    tmp_base.set_index(col, inplace=True)
    
    merge['no_fraud'] = (merge[f'{col}_mean']==0)
    tmp_base['no_fraud_sum'] = merge.groupby(col)['no_fraud'].sum()
    tmp_base[f'{col}_monthly_fraud_prob_max_min_diff'] = merge.groupby(col)[f'{col}_mean'].max() - merge.groupby(col)[f'{col}_mean'].min()
    tmp_base[f'{col}_monthly_fraud_prob_max_min_diff'] = merge.groupby(col)[f'{col}_mean'].max() - merge.groupby(col)[f'{col}_mean'].min()
    tmp_base[f'{col}_monthly_fraud_prob_max_min_ratio'] = merge.groupby(col)[f'{col}_mean'].max() / merge.groupby(col)[f'{col}_mean'].min()
    tmp_base[f'{col}_monthly_fraud_prob_std'] = merge.groupby(col)[f'{col}_mean'].std()
    
    tmp_base[f'{col}_monthly_fraud_ratio_max_min_diff'] = merge.groupby(col)['ratio'].max() - merge.groupby(col)['ratio'].min()
    tmp_base[f'{col}_monthly_fraud_ratio_max_min_diff'] = merge.groupby(col)['ratio'].max() - merge.groupby(col)['ratio'].min()
    tmp_base[f'{col}_monthly_fraud_ratio_max_min_ratio'] = merge.groupby(col)['ratio'].max() / merge.groupby(col)['ratio'].min()
    tmp_base[f'{col}_monthly_fraud_ratio_std'] = merge.groupby(col)['ratio'].std()
    
    # Weighted Mean
    merge['weight'] = merge['DT-M'].map(weight_map)
    merge[f'{col}_weight'] = merge['weight'] * merge[f'{col}_mean']
    tmp_base[f'{col}_weighted_mean'] = merge.groupby(col)[f'{col}_weight'].sum() / merge.groupby(col)['weight'].sum()
    tmp_base[f'{col}_fraud_ratio_trend'] = tmp_base[f'{col}_weighted_mean'] / tmp_base[f'{col}_all_mean']
    
    tmp_base.fillna(0, inplace=True)
    
    df_feature = base.merge(tmp_base, how='left', on=col)

In [46]:
df_feature

,card2,card2_all_mean,no_fraud_sum,card2_monthly_fraud_prob_max_min_diff,card2_monthly_fraud_prob_max_min_ratio,card2_monthly_fraud_prob_std,card2_monthly_fraud_ratio_max_min_diff,card2_monthly_fraud_ratio_max_min_ratio,card2_monthly_fraud_ratio_std,card2_weighted_mean,card2_fraud_ratio_trend
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,404.0,0.065118,0.0,0.105658,8.480581,0.037968,1.622566,8.480581,0.583074,0.056574,0.868793
2,490.0,0.024014,0.0,0.009451,1.479135,0.003939,0.393568,1.479135,0.164034,0.024847,1.034701
3,567.0,0.016295,0.0,0.015794,3.009032,0.005716,0.969295,3.009032,0.350819,0.018154,1.114141
4,514.0,0.046214,0.0,0.045597,2.818577,0.017529,0.986640,2.818577,0.379299,0.050850,1.100312
5,555.0,0.022836,0.0,0.015664,2.155749,0.005297,0.685947,2.155749,0.231948,0.024189,1.059251
6,360.0,0.010402,0.0,0.004825,1.632060,0.001762,0.463861,1.632060,0.169411,0.010189,0.979561
7,490.0,0.024014,0.0,0.009451,1.479135,0.003939,0.393568,1.479135,0.164034,0.024847,1.034701
8,100.0,0.012814,0.0,0.007922,1.994612,0.003175,0.618218,1.994612,0.247755,0.012550,0.979433
9,111.0,0.021641,0.0,0.013934,1.969402,0.004659,0.643833,1.969402,0.215270,0.022759,1.051639
